In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [2]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name} with length {length} bytes'.format(name =fn, length = len(uploaded[fn])))

Saving finalriver.csv to finalriver.csv
User uploaded file "finalriver.csv with length 598 bytes


In [3]:
df_river = pd.read_csv("finalriver.csv")
print(df_river)

         Date       State       Districts  Level(m)
0    1/1/2023  Tamil nadu  Kanniya Kumari      1.61
1    1/2/2023  Tamil nadu  Kanniya Kumari      1.61
2    1/3/2023  Tamil nadu  Kanniya Kumari      1.62
3    1/4/2023  Tamil nadu        Chnennai      1.63
4    1/5/2023  Tamil nadu        Chnennai      1.63
5    1/6/2023  Tamil nadu        Chnennai      1.63
6    1/7/2023  Tamil nadu  Tiruvannamalai      1.63
7    1/8/2023  Tamil nadu  Tiruvannamalai      1.63
8    1/9/2023  Tamil nadu  Tiruvannamalai      1.63
9   1/10/2023  Tamil nadu  Tiruvannamalai      1.63
10  1/11/2023  Tamil nadu      Coimbatore      1.64
11  1/12/2023  Tamil nadu      Coimbatore      1.61
12  1/13/2023  Tamil nadu      Coimbatore      1.65
13  1/14/2023  Tamil nadu      Coimbatore      1.67
14  1/15/2023  Tamil nadu      Coimbatore      1.56


In [4]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name} with length {length} bytes'.format(name =fn, length = len(uploaded[fn])))

Saving final-rainfall.csv to final-rainfall.csv
User uploaded file "final-rainfall.csv with length 668 bytes


In [5]:
df_rain = pd.read_csv("final-rainfall.csv")
print(df_rain)

         Date       State       Districts  Current Rainfall(mm)  \
0    1/1/2023  Tamil nadu  Kanniya Kumari                   0.0   
1    1/2/2023  Tamil nadu  Kanniya Kumari                   0.0   
2    1/3/2023  Tamil nadu  Kanniya Kumari                   0.0   
3    1/4/2023  Tamil nadu        Chnennai                   0.0   
4    1/5/2023  Tamil nadu        Chnennai                   0.6   
5    1/6/2023  Tamil nadu        Chnennai                   0.8   
6    1/7/2023  Tamil nadu  Tiruvannamalai                   0.4   
7    1/8/2023  Tamil nadu  Tiruvannamalai                   0.5   
8    1/9/2023  Tamil nadu  Tiruvannamalai                   0.2   
9   1/10/2023  Tamil nadu  Tiruvannamalai                   0.3   
10  1/11/2023  Tamil nadu      Coimbatore                   0.1   
11  1/12/2023  Tamil nadu      Coimbatore                   0.4   
12  1/13/2023  Tamil nadu      Coimbatore                   0.5   
13  1/14/2023  Tamil nadu      Coimbatore                   0.

In [8]:
df = pd.merge(df_rain, df_river, on=['Date', 'State', 'Districts'])

In [9]:
df.dropna(inplace=True)

In [10]:
flood_threshold = 1.5

In [11]:
df['Flood'] = np.where(df['Level(m)'] > flood_threshold, 1, 0)

In [13]:
X = df[['Current Rainfall(mm)', 'Cumulative Rainfall(mm)']].values
y = df['Flood'].values

In [14]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [16]:
print(X_train)

[[0.         0.36091954]
 [1.         0.18390805]
 [0.25       0.08045977]
 [0.         0.11494253]
 [0.         0.09195402]
 [0.         0.63218391]
 [0.75       0.45977011]
 [0.625      0.12643678]
 [0.125      0.08045977]
 [0.625      0.51724138]
 [0.         0.26436782]
 [0.5        0.17241379]]


In [29]:
# Reshape input data for LSTM
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [30]:
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')  # Sigmoid activation for binary classification
])

In [31]:
# Compile the model
#optimizer = Adam(learning_rate=0.001)
#model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='mean_squared_error')

In [32]:
# Train-test split# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=5, batch_size=2, validation_data=(X_test_reshaped, y_test))

Epoch 1/5
6/6 [==============================] - 8s 350ms/step - loss: 0.2455 - val_loss: 0.2394
Epoch 2/5
6/6 [==============================] - 0s 18ms/step - loss: 0.2345 - val_loss: 0.2269
Epoch 3/5
6/6 [==============================] - 0s 13ms/step - loss: 0.2195 - val_loss: 0.2101
Epoch 4/5
6/6 [==============================] - 0s 17ms/step - loss: 0.2021 - val_loss: 0.1869
Epoch 5/5
6/6 [==============================] - 0s 16ms/step - loss: 0.1754 - val_loss: 0.1548


In [33]:
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)

1/1 [==============================] - 1s 1s/step


In [36]:
# Print predictions for all available data
for i in range(len(y_pred)):
    predicted_label = "Flood" if y_pred[i][0] == 1 else "No Flood"
    print(f"Prediction {i+1}: {predicted_label}")

Prediction 1: Flood
Prediction 2: Flood
Prediction 3: Flood


In [39]:
# Save the model to disk
model.save("C://Users//indhr//OneDrive//Desktop//dataset//flood_prediction_model.h5")
